In [1]:
from Bio.PDB import *
import glob
CaMKII = glob.glob("input.loop3/CaMKII_loop3_*.pdb")
holoCaM = glob.glob("input.loop3/holoCaM_loop3_*.pdb")
Ng=glob.glob("input.loop3/Ng-*_loop3_*.pdb")
loop3Size = 176

files = glob.glob("output.loop3/*.txt")
charge = dict()
for chgFile in files:
    prot = chgFile.split('/')[1].split('.')[1]
    with open(chgFile,"r") as file1:
        for line in file1:
            line = line.strip().split()
            charge[(prot,line[0])] = line[1]

output = open("distData.txt", 'w+')


def count(gen):
    return sum(1 for _ in gen.get_atoms())

parser = PDBParser()
structure = parser.get_structure('loop3', 'input.loop3/CaMKII_loop3_0.0.pdb')
if count(structure) != loop3Size:
    print("Error in atom number!\n")
    raise SystemExit(1)

for i in structure.get_atoms():
    if i.parent.get_resname() == ' CA':
        calAtom = i


In [12]:
f=list(structure.get_residues())[0]

In [18]:
map(lambda x: print(x), f.get_atoms())

In [25]:
fatoms = [ x for x in f.get_atoms()]

In [28]:
fatoms[3]

<Atom H01>

In [30]:
f.full_id

('loop3', 0, 'X', (' ', 92, ' '))